In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Using Vertex AI Knowledge Engine with Pinecone

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/knowledge_engine_pinecone.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fvertex-ai-samples%2Fmain%2Fnotebooks%2Fcommunity%2Fmodel_garden%2Fknowledge_engine_pinecone.ipynb"">
      <img width="32px" src="https://cloud.google.com/ml-engine/images/colab-enterprise-logo-32px.png" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td>    
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/knowledge_engine_pinecone.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Workbench
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/knowledge_engine_pinecone.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>

## Overview
This notebook demonstrates how to bring your Pinecone instance for use with the Vertex AI Knowledge Engine to serve as the vector store of choice.

## Get Started

### Install Vertex AI SDK for Python and other required packages

In [ ]:
!pip3 install --force-reinstall google-cloud-aiplatform

In [ ]:
!pip3 install --upgrade google-cloud
!pip3 install --upgrade google-cloud-secret-manager
!pip install --upgrade "pinecone-client[grpc]"

### Restart runtime (Colab only)
To use the newly installed packages, you must restart the runtime on Google Colab.

In [ ]:
import sys

if "google.colab" in sys.modules:

    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

### Authenticate your notebook environment (Colab only)
Authenticate your environment on Google Colab.

In [ ]:
import sys

if "google.colab" in sys.modules:

    from google.colab import auth

    auth.authenticate_user()

### Set Google Cloud project information

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com). Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [ ]:
PROJECT_ID = ""  # @param {type:"string"}

PROJECT_NUMBER = ""  # @param {type:"string"}

LOCATION = ""  # @param {type:"string"}

### Initialize Vertex AI SDK for Python

In [ ]:
import vertexai
from vertexai.preview import rag
from vertexai.preview.generative_models import GenerativeModel, Tool

vertexai.init(project=PROJECT_ID, location=LOCATION)

### Setup Pinecone Instance

[**OPTIONAL**]

In this section, we have some helper methods to help you setup your Pinecone instance. This section is not required if you already have a Pinecone instance ready to use.

#### Initialize the Pinecone Python client

In [ ]:
# Set API Key
# Copy this value from your pinecone.io console
PINECONE_API_KEY = ""  # @param {type:"string"}


from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec, PodSpec

pc = Pinecone(api_key=PINECONE_API_KEY)

#### Create a Pinecone instance

Use the below section to create Pinecone indexes with a spec of your choice (serverless or pod). Read more about Pinecone indexes [here](https://docs.pinecone.io/guides/indexes/create-an-index).

##### Serverless Index

In [ ]:
# Index Configs
INDEX_NAME = ''  # @param {type:"string"}

# Choose a distance metric
DISTANCE_METRIC = 'cosine'  # @param ["cosine", "euclidean", "dotproduct"] {allow-input: true}

# This number should match the dimension size of the embedding model you choose
# for your RAG Corpus.
EMBEDDING_DIMENSION_SIZE = 768  # @param {"type":"number","placeholder":"768"}

CLOUD_PROVIDER = 'gcp'  # @param ["gcp", "aws", "azure"] {allow-input: true}
# Choose the right region for your cloud provider of choice
# Refer https://docs.pinecone.io/guides/indexes/understanding-indexes#cloud-regions
CLOUD_REGION = "us-central1"  # @param {type:"string"}



# Create the index
pc.create_index(
  name=INDEX_NAME,
  dimension=EMBEDDING_DIMENSION_SIZE,
  metric=DISTANCE_METRIC,
  spec=ServerlessSpec(
    cloud=CLOUD_PROVIDER,
    region=CLOUD_REGION
  ),
  deletion_protection="disabled"
)

##### Pod-based Index


In [ ]:
# Index Configs
INDEX_NAME = ''  # @param {type:"string"}

# Choose a distance metric
DISTANCE_METRIC = 'cosine'  # @param ["cosine", "euclidean", "dotproduct"] {allow-input: true}

# This number should match the dimension size of the embedding model you choose
# for your RAG Corpus.
EMBEDDING_DIMENSION_SIZE = 768  # @param {"type":"number","placeholder":"768"}

# Choose the right environment for your cloud provider of choice
# Refer https://docs.pinecone.io/guides/indexes/understanding-indexes#pod-environments
ENVIRONMENT = "us-central1-gcp"  # @param {type:"string"}

# Choose the pod type
# Refer to https://docs.pinecone.io/guides/indexes/understanding-indexes#pod-based-indexes
POD_TYPE = "p1.x1"  # @param {type:"string"}

# Explore all the parameters you can play with for creating a pod index by
# following this page:
# https://docs.pinecone.io/reference/api/2024-07/control-plane/create_index
pc.create_index(
  name=INDEX_NAME,
  dimension=EMBEDDING_DIMENSION_SIZE,
  metric=DISTANCE_METRIC,
  spec=PodSpec(
    environment=ENVIRONMENT,
    pod_type=POD_TYPE,
    pods=1,
    metadata_config={
        "indexed": ["file_id"]  # This field is required for pod-based indexes.
    }
  ),
  deletion_protection="disabled"
)

### Setup Secret Manager

[**OPTIONAL**]

This section helps you add your Pinecone API key to your Google Cloud Secret Manager. This section is not required is you already have a secret with the API key ready to use.

In [ ]:
# GCP project ID and the Pinecone API key will be used from the above sections.
# Choose your secret ID
SECRET_ID = ""   # @param {type:"string"}

In [ ]:
from google.cloud import secretmanager

client = secretmanager.SecretManagerServiceClient()

# Build the resource name of the parent project.
project_parent = f"projects/{PROJECT_ID}"
secret_payload = PINECONE_API_KEY.encode("UTF-8")


# Create the secret.
secret_creation_response = client.create_secret(
    request={
        "parent": project_parent,
        "secret_id": SECRET_ID,
        "secret": {
            "replication": {"automatic": {}},
        },
    }
)

# Add API key to the secret payload.
# Build the resource name of the parent secret.
secret_parent = client.secret_path(PROJECT_ID, SECRET_ID)
secret_version_response = client.add_secret_version(
    request={"parent": secret_parent, "payload": {"data": secret_payload}}
)

print(f"Created secret and added first version: {secret_version_response.name}")

##### Give access to Knowledge Engine service account permissions to the secret

In [ ]:
def add_secret_accessor_permission(secret_resource_name: str, service_account: str):
    from google.iam.v1 import policy_pb2

    iam_member = f"serviceAccount:{service_account}"

    # Get the current IAM policy.
    policy = client.get_iam_policy(request={"resource": secret_resource_name})

    # Add the new binding.
    binding = policy_pb2.Binding()
    binding.role = "roles/secretmanager.secretAccessor"
    binding.members.append(iam_member)
    policy.bindings.append(binding)

    # Update the IAM policy.
    client.set_iam_policy(
        request={"resource": secret_resource_name, "policy": policy}
    )

    print(f"Updated IAM policy for secret: {secret_resource_name}")

**NOTE**: Skip this below section if this is your first RAG Corpus in the project. This is because your Knowledge Engine service account only gets provisioned during your first resource creation and hence you cannot assign it permissions. The later sections will help you add this after creating your RAG Corpus.

In [ ]:
# Construct your Knowledge Engine service account name
# Do not update this string since this is the name assigned to your service
# account.
SERVICE_ACCOUNT = (
    f"service-{PROJECT_NUMBER}@gcp-sa-vertex-rag.iam.gserviceaccount.com"
)

add_secret_accessor_permission(
    secret_resource_name=secret_parent, service_account=SERVICE_ACCOUNT
)

## Create your RAG Corpus

### First RAG Corpus Only

If this is the first RAG Corpus in your project, you might not be able to provide the Knowledge Engine service account access to your secret resource. So this section first creates a RAG Corpus with an empty Pinecone config. With this call, the service account for your project is provisioned.

Next, it assigns the service account permissions to read your secret. Finally, it updates your RAG Corpus with the Pinecone index name and the secret resource name.

##### Create your RAG Corpus without Pinecone information

In [ ]:
# Name your corpus
DISPLAY_NAME = ""  # @param  {type:"string"}

# Start with empty Pinecone config.
vector_db = rag.Pinecone()

# Create RAG Corpus
rag_corpus = rag.create_corpus(display_name=DISPLAY_NAME, vector_db=vector_db)
print(f"Created RAG Corpus resource: {rag_corpus.name}")

##### Grant your Knowledge Engine service account access to your API key secret

In [ ]:
# Construct your Knowledge Engine service account name
# Do not update this string since this is the name assigned to your service
# account.
SERVICE_ACCOUNT = (
    f"service-{PROJECT_NUMBER}@gcp-sa-vertex-rag.iam.gserviceaccount.com"
)

from google.cloud import secretmanager

client = secretmanager.SecretManagerServiceClient()

secret_parent = client.secret_path(PROJECT_ID, SECRET_ID)
add_secret_accessor_permission(
    secret_resource_name=secret_parent, service_account=SERVICE_ACCOUNT
)

##### Call the UpdateRagCorpus API to add the Pinecone index name and API key secret to your RAG Corpus

In [ ]:
# Full resource name of your created secret version.
# Format: projects/{PROJECT_ID}/secrets/{SECRET_ID}/versions/{VERSION_ID}
SECRET_RESOURCE_NAME = ""  # @param  {type:"string"}

# Name of your created Pinecone Index
PINECONE_INDEX_NAME = ""  # @param  {type:"string"}

# Construct your updated Pinecone config.
vector_db = rag.Pinecone(
    index_name=PINECONE_INDEX_NAME, api_key=SECRET_RESOURCE_NAME
    )

updated_rag_corpora = rag.update_corpus(
    corpus_name=rag_corpus.name, vector_db=vector_db
    )
print(f"Updated RAG Corpus: {rag_corpus.name}")

### Second RAG Corpus Onwards

In this case, since your service account is already generated, you can directly grant it permissions to access your secret resource containing the Pinecone API key as covered by the steps in the Setup Secret Manager section.





In [ ]:
# Full resource name of your created secret version.
# Format: projects/{PROJECT_ID}/secrets/{SECRET_ID}/versions/{VERSION_ID}
SECRET_RESOURCE_NAME = ""  # @param  {type:"string"}

# Name of your created Pinecone Index
PINECONE_INDEX_NAME = ""  # @param  {type:"string"}

# Name your corpus
DISPLAY_NAME = ""  # @param  {type:"string"}

# Construct your Pinecone config.
vector_db = rag.Pinecone(
    index_name=PINECONE_INDEX_NAME, api_key=SECRET_RESOURCE_NAME
)

# Create RAG Corpus
rag_corpus = rag.create_corpus(display_name=DISPLAY_NAME, vector_db=vector_db)
print(f"Created RAG Corpus resource: {rag_corpus.name}")

## Upload a file to your RAG Corpus

In [ ]:
%%writefile test.txt

Here's a demo for testing RAG upload.

In [ ]:
rag_file = rag.upload_file(
    corpus_name=rag_corpus.name,
    path="test.txt",
    display_name="test.txt",
    description="my test",
)

print(f"Uploaded file to resource: {rag_file.name}")

## Import files from Google Cloud Storage into your RAG Corpus

Remember to grant "Viewer" access to the Knowledge Engine service account for your Google Cloud Storage bucket.



In [ ]:
GCS_BUCKET = ""  # @param {type:"string"}

response = await rag.import_files_async(  # noqa: F704
    corpus_name=rag_corpus.name,
    paths=[GCS_BUCKET],
    chunk_size=512,
    chunk_overlap=50,
)

In [ ]:
# Check the files just imported. It may take a few seconds to process the imported files.
list(rag.list_files(corpus_name=rag_corpus.name))

## Import files from Google Drive
Eligible paths can be https://drive.google.com/drive/folders/{folder_id} or https://drive.google.com/file/d/{file_id}.

Remember to grant "Viewer" access to the Knowledge Engine service account for your Drive folder/files.

In [ ]:
FILE_ID = ""  # @param {type:"string"}
FILE_PATH = f"https://drive.google.com/file/d/{FILE_ID}"

rag.import_files(
    corpus_name=rag_corpus.name,
    paths=[FILE_PATH],
    chunk_size=1024,
    chunk_overlap=100,
)

In [ ]:
# Check the files just imported. It may take a few seconds to process the imported files.
list(rag.list_files(corpus_name=rag_corpus.name))

## Use your RAG Corpus to add context to your Gemini queries


In [ ]:
rag_resource = rag.RagResource(
    rag_corpus=rag_corpus.name,
)

rag_retrieval_tool = Tool.from_retrieval(
    retrieval=rag.Retrieval(
        source=rag.VertexRagStore(
            rag_resources=[rag_resource],  # Currently only 1 corpus is allowed.
            similarity_top_k=10,
            vector_distance_threshold=0.4,
        ),
    )
)

rag_model = GenerativeModel(
    "gemini-1.5-pro-001",
    tools=[rag_retrieval_tool]
)

In [ ]:
GENERATE_CONTENT_PROMPT = ""  # @param {type:"string"}

response = rag_model.generate_content(GENERATE_CONTENT_PROMPT)

response

## Cleaning Up

Clean up the resources created in this notebook.

In [ ]:
delete_rag_corpus = False  # @param {type:"boolean"}

if delete_rag_corpus:
    rag.delete_corpus(name=rag_corpus.name)

## API reference

For more details on RAG corpus/file management and detailed support please visit https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/rag-api